# Helpful links:
Database connection:
https://docs.intersystems.com/irislatest/csp/docbook/DocBook.UI.Page.cls?KEY=BPYNAT_pyapi

SQL:
https://docs.intersystems.com/irislatest/csp/docbook/DocBook.UI.Page.cls?KEY=RSQL_createtable

Data Types:
https://docs.intersystems.com/irislatest/csp/docbook/DocBook.UI.Page.cls?KEY=RSQL_datatype


<b> Make sure to start the IRIS database before running this file. A quick way to do that: </b> \
docker run -d --name iris-comm -p 1972:1972 -p 52773:52773 -e IRIS_PASSWORD=demo -e IRIS_USERNAME=demo intersystemsdc/iris-community:latest

# Loading the data to a dataframe
The data used here is the D_ICD_DIAGNOSES table from the Medical Information Mart for Intensive Care (MIMIC)-IV Demo database \
<i>" Medical Information Mart for Intensive Care (MIMIC)-IV database is comprised of deidentified electronic health records for patients admitted to the Beth Israel Deaconess Medical Center " </i> \
Johnson, A., Bulgarelli, L., Pollard, T., Horng, S., Celi, L. A., & Mark, R. (2023). MIMIC-IV Clinical Database Demo (version 2.2). PhysioNet. https://doi.org/10.13026/dp1f-ex47.

In [82]:
import pandas as pd

In [83]:
diagnoses = pd.read_csv("D_ICD_DIAGNOSES.csv")

In [84]:
diagnoses.head()

,row_id,icd9_code,short_title,long_title
0,1,01716,Erythem nod tb-oth test,Erythema nodosum with hypersensitivity reactio...
1,2,01720,TB periph lymph-unspec,"Tuberculosis of peripheral lymph nodes, unspec..."
2,3,01721,TB periph lymph-no exam,"Tuberculosis of peripheral lymph nodes, bacter..."
3,4,01722,TB periph lymph-exam unk,"Tuberculosis of peripheral lymph nodes, bacter..."
4,5,01723,TB periph lymph-micro dx,"Tuberculosis of peripheral lymph nodes, tuberc..."


# IRIS database operations

In [ ]:
!pip install intersystems_irispython-3.2.0-py3-none-any.whl
import iris
import time

## Database connection settings

In [87]:
namespace="USER"
port = 1972
hostname="localhost"
connection_string = f"{hostname}:{port}/{namespace}"
username = "demo"
password = "demo"

In [108]:
# Note: Ideally conn and cursor should be used with context manager or with try-execpt-finally 
conn = iris.connect(connection_string, username, password)
cursor = conn.cursor()

## Creating a new table

In [89]:
tableName = "Demo.Diagnoses"
tableDefinition = "(row_id INTEGER, icd9_code VARCHAR(255), short_title VARCHAR(255), long_title VARCHAR(''))"

In [90]:
try:
    cursor.execute(f"DROP TABLE {tableName}")  
except:
    pass
cursor.execute(f"CREATE TABLE {tableName} {tableDefinition}")

0

## Adding a batch of data

In [91]:
## batch update
sql = "Insert into Demo.Diagnoses (row_id, icd9_code,short_title,long_title) values (?, ?, ?,?)"
params = [('0', '0', "entered from batch update 1","entered from batch update 1"), ('0', '0','entered from batch update 2',"entered from batch update 2"), ('0', '0',"entered from batch update 3", "entered from batch update 3")]
cursor.executemany(sql, params) 

3

## Adding data one row at a time by looping over the dataframe

In [92]:
##looping through dataframe and adding all the data to IRIS table
sql = "Insert into Demo.Diagnoses (row_id, icd9_code,short_title,long_title) values (?, ?, ?,?)"
start_time = time.time()
for index,row in diagnoses.iterrows():
    cursor.execute(sql, [row.row_id,row.icd9_code,row.short_title,row.long_title])
end_time = time.time()
print(f"time taken to add {len(diagnoses)} entries: {end_time-start_time} seconds")

time taken to add 14567 entries: 3.831266164779663 seconds


## Reading from the table

In [93]:
##fetching data from database
cursor.execute("select * from Demo.Diagnoses")
fetched_data = cursor.fetchmany(5)
for row in fetched_data:
    print(row)

[0, '0', 'entered from batch update 1', 'entered from batch update 1']
[0, '0', 'entered from batch update 2', 'entered from batch update 2']
[0, '0', 'entered from batch update 3', 'entered from batch update 3']
[1, '01716', 'Erythem nod tb-oth test', 'Erythema nodosum with hypersensitivity reaction in tuberculosis, tubercle bacilli not found by bacteriological or histological examination, but tuberculosis confirmed by other methods [inoculation of animals]']
[2, '01720', 'TB periph lymph-unspec', 'Tuberculosis of peripheral lymph nodes, unspecified']


In [94]:
##close the connection
cursor.close()
conn.commit()
conn.close()